In [ ]:
j = 0
x = None
codes = []
return_mat = np.zeros((11602, 1271))
j = 0
for i in code_shares:
    price_matrix= sp_data[i].astype('float')
    price_matrix = np.array(price_matrix)
    returns = np.log(price_matrix[1:]) - np.log(price_matrix[0:-1])
    if np.where(returns<-5)[0].shape[0] == 0:
        codes.append(i)
        returns[np.isnan(returns)]=-5
        return_mat[:, j] = returns
        j += 1
df_return = pd.DataFrame(data=return_mat, columns=codes)
df_return.to_csv('returns127.csv')

ffdata = pd.read_csv('./FF_Factors_daily.csv')
start_date = 19750102
end_date = 20201231
mask = (ffdata['date'] >= start_date) & (ffdata['date'] <= end_date)
ffdata = ffdata.loc[mask]
ffdata.to_csv('ffdata_11062.csv')

In [10]:
import json
import mgarch
import os
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cvxopt import matrix
from cvxopt.blas import dot 
from cvxopt import solvers
solvers.options['show_progress'] = False


def load_json(path):
    with open(path, 'rb') as f:
        json_dict = json.load(f)
    return json_dict


def find_first_800(sp_data_matrix, num):
    posts = []
    trading_days = sp_data_matrix.shape[0]
    for i in range(trading_days):
        num_eff = np.sum(sp_data_matrix[i, 2:]>-5)
        if num_eff == num:
            posts.append(i)
    return posts[0]


def get_sub_mat(df_return, st_idx, code_shares, sp_data, sam_size):
    indx_length = sam_size
    num_lim = indx_length + 21
    ed_idx = st_idx + num_lim
    code_ls = list(df_return.columns)[1:]
    code_used = []
    res_mat = []
    res_dict = {}
    res_pre_dict = {}
    market_cap_dict = {}
    for code_i in code_ls:
        res = df_return[code_i][st_idx:ed_idx]
        eff_num = np.sum(res>-5)
        if eff_num == num_lim:
            cap_i = code_shares[code_i] * sp_data[code_i][st_idx+indx_length]# error: -1
            res_ls  = list(res)
            res_mat.append(res_ls[0:indx_length])
            code_used.append(code_i)
            res_dict[code_i] = res_ls[0:indx_length]
            res_pre_dict[code_i] = res_ls[indx_length:indx_length+21]
            market_cap_dict[code_i] = cap_i
    return res_mat, code_used, res_dict, res_pre_dict, market_cap_dict


def get_N(res_mat, code_used, res_dict, res_pre_dict, market_cap_dict, N):
    pos = np.where(np.corrcoef(np.array(res_mat))>0.95)
    if len(pos)>1:
        line_num, row_num = pos
        line_num = line_num.tolist()
        row_num = row_num.tolist()
        code_used_ori = copy.deepcopy(code_used)
        code_used = set(code_used)
        for i in range(len(line_num)):
            for j in range(len(row_num)):
                if  i > j:
                    stock_i = code_used_ori[line_num[i]]
                    stock_j = code_used_ori[row_num[j]]
                    cap_i = market_cap_dict[stock_i]
                    cap_j = market_cap_dict[stock_j]
            
                    if  cap_i>=cap_j and cap_i in code_used:
                        code_used.remove(stock_j)
                    elif cap_i<cap_j and cap_i in code_used:
                        code_used.remove(stock_i)
    
    code_cap = {}
    for code_i in code_used:
        code_cap[code_i] =  market_cap_dict[code_i]
        
    code_cap_sorted = sorted(code_cap.items(), key=lambda x: x[1], reverse=True)
    code_N = [i[0] for i in code_cap_sorted[0:N]]
    
    res_mat_new = []
    res_pre_new = []
    for code_i in code_N:
        res_mat_new.append(res_dict[code_i])
        res_pre_new.append(res_pre_dict[code_i])
    return res_mat_new, res_pre_new


def get_rolls(code_shares, sp_data, df_return, last_num, N, num, sam_size):
    st_id = find_first_800(df_return.to_numpy(), num)
    roll_len = sam_size+21
    j = 0
    all_rolls_res = 1
    while st_id+sam_size+21 <= last_num:
        res_mat, code_used, res_dict, res_pre_dict, market_cap_dict = get_sub_mat(df_return,
                                                st_id, code_shares, sp_data, sam_size)
        res_mat_new, res_pre_new = get_N(res_mat, code_used, 
                               res_dict, res_pre_dict, market_cap_dict, N)
        res_mat_new = np.array(res_mat_new).transpose()
        res_pre_new = np.array(res_pre_new).transpose()
        res_all = np.vstack((res_mat_new, res_pre_new))
        st_id += 21
        if all_rolls_res is  1:
            all_rolls_res = res_all
        else:
            all_rolls_res = np.vstack((all_rolls_res, res_all))
        
    return all_rolls_res


def get_return(weights, res_pre):
    weights = np.matrix(weights)
    res_pre = np.matrix(res_pre)
    return (weights*res_pre).tolist()[0]
    

def average_weights(n):
    return np.matrix([1/n for i in range(n)])


def get_ws_cov(cov, no_short=True):
    if no_short:
        N = cov.shape[0]
        P = matrix(cov)
        q = matrix(np.zeros((N, 1)))
        G  = matrix(-np.identity(N))
        h = matrix(np.zeros((N, 1)))
        A = matrix(1.0, (1, N))
        b = matrix(1.0)
        sol = solvers.qp(P, q, G, h, A, b)
        ws = np.matrix(sol['x']).T
    else:
        cov_mat = np.matrix(cov)
        N = cov_mat.shape[0]
        I = np.matrix(np.ones(N)).T
        ws = (cov_mat.I * I)/(I.T*cov_mat.I*I)
        ws = ws.T
    return ws  


def get_sam_res(code_shares, sp_data, df_return, last_num, N, num, sam_size):
    st_id = find_first_800(df_return.to_numpy(), num)
    res_ew = []
    res_sample_cov = []
    while st_id + sam_size+21 <= last_num:
        res_mat, code_used, res_dict, res_pre_dict, market_cap_dict = get_sub_mat(df_return,
                                                                st_id, code_shares, sp_data, sam_size)
        res_mat_new, res_pre_new = get_N(res_mat, code_used, 
                               res_dict, res_pre_dict, market_cap_dict, N)
        st_id += 21
        avg_ws = average_weights(N)
        avg_res_i = get_return(avg_ws, res_pre_new)
        res_ew += avg_res_i
        sam_cov = np.cov(np.array(res_mat_new))
        samp_ws = get_ws_cov(sam_cov, no_short=False)
        sam_res_i = get_return(samp_ws,  res_pre_new)
        res_sample_cov += sam_res_i
    return res_ew, res_sample_cov


def indicators(res):
    AV= round(np.mean(res)*252*100, 5)
    SD = round(np.cov(res)**0.5*252**0.5*100, 5)
    IR = round(AV/SD, 5)
    return AV, SD, IR


def read_dcc_res(path):
    res_dccnl = pd.read_csv(path)
    res_dccnl= res_dccnl['x']
    res_dccnl = list(res_dccnl)
    return res_dccnl

def res_line(res):
    portfolio_res = [1]
    a = 1
    for i in res:
        a = a*(1+i)
        portfolio_res.append(a)
    return portfolio_res

def get_dcc_res1(root_dir, rolls, N, no_short):
    file_ls = os.listdir(root_dir)
    length = len(file_ls)
    res_dccnl = []
    for i in range(length):
        file_path = os.path.join(root_dir, '{}.csv'.format(i+1))
        cov_mat = pd.read_csv(file_path)
        cov_mat = np.matrix(cov_mat.to_numpy())
        #st_idx = i*1281
        #ed_idx = i*1281+1260
        #res_mat = rolls[st_idx:ed_idx, 1:N+1]
        #cov_mat =  np.cov(np.array(res_mat).transpose())
        ws_i = get_ws_cov(cov_mat, no_short=no_short)
        st_idx = (i+1)*1281-21
        ed_idx = (i+1)*1281
        res_pre_i = rolls[st_idx:ed_idx, 1:N+1]
        res_pre_i = np.matrix(res_pre_i).T
        res_i = get_return(ws_i, res_pre_i)
        res_dccnl += res_i
    return res_dccnl

# 1 load_data and get return rates

In [3]:
code_shares = load_json('./code_shares.json')
sp_data = pd.read_csv('./SP1273_19750103_20201231.csv')
df_return = pd.read_csv('./returns1271.csv')

# 2 get all rolling res data

In [ ]:
all_rolls_res = get_rolls(code_shares, sp_data, df_return, 11602, 500, 800, 630)

In [315]:
all_rolls_res1 = np.around(all_rolls_res, decimals=6, out=None)
col_names = []
for i in range(500):
    col_names.append('asset{}'.format(i))
df_rolls = pd.DataFrame(data=all_rolls_res1, columns=col_names)
df_rolls.to_csv('df_rolls_500_630.csv')

# 3 get portfolio universe

In [4]:
rolls = pd.read_csv('./df_rolls.csv')
rolls = rolls.to_numpy()

# N=100

In [148]:
avg_res_100, samp_res_100 = get_sam_res(code_shares, sp_data, df_return, 11602, 100, 800, 1260)

In [149]:
root_dir = './cov_100_dcc/'
res_dccnl_100 = get_dcc_res1(root_dir, rolls, 100, no_short=False)

In [150]:
AV_avg, SD_avg, IR_avg = indicators(avg_res_100)
AV_sam, SD_sam, IR_sam = indicators(samp_res_100)
AV_DCCNL, SD_DCCNL, IR_DCCNL = indicators(res_dccnl_100)
print('|','1/N', AV_avg, '|', SD_avg, '|', IR_avg, '|')
print('|','sample', AV_sam, '|', SD_sam, '|', IR_sam,'|')
print('|', 'DCCNL', AV_DCCNL, '|', SD_DCCNL, '|', IR_DCCNL,'|')

| 1/N 5.19048 | 20.65605 | 0.25128 |
| sample -692.8167 | 3127.03814 | -0.22156 |
| DCCNL 7.42985 | 13.78978 | 0.53879 |


# N = 200

In [153]:
avg_res_200, samp_res_200 = get_sam_res(code_shares, sp_data, df_return, 11602, 200, 800, 1260)

In [154]:
root_dir = './cov_200_dcc/'
res_dccnl_200 = get_dcc_res1(root_dir, rolls, 200, no_short=False)

In [155]:
AV_avg, SD_avg, IR_avg = indicators(avg_res_200)
AV_sam, SD_sam, IR_sam = indicators(samp_res_200)
AV_DCCNL, SD_DCCNL, IR_DCCNL = indicators(res_dccnl_200)
print('|','1/N', AV_avg, '|', SD_avg, '|', IR_avg, '|')
print('|','sample', AV_sam, '|', SD_sam, '|', IR_sam,'|')
print('|', 'DCCNL', AV_DCCNL, '|', SD_DCCNL, '|', IR_DCCNL,'|')

| 1/N 5.58737 | 20.94381 | 0.26678 |
| sample -550.53041 | 2487.3752 | -0.22133 |
| DCCNL 9.77188 | 12.96225 | 0.75387 |


# N=500

In [157]:
avg_res_500, samp_res_500 = get_sam_res(code_shares, sp_data, df_return, 11602, 500, 800, 1260)

In [158]:
root_dir = './cov_500_dcc/'
res_dccnl_500 = get_dcc_res1(root_dir, rolls, 500, no_short=False)

In [159]:
print(indicators(avg_res_500))
print(indicators(samp_res_500))
print(indicators(res_dccnl_500))

(6.82722, 21.95549, 0.31096)
(-286.08358, 1302.92072, -0.21957)
(8.1287, 12.13695, 0.66975)


# 2 static model

## 2.1 five factor model

In [101]:
def get_rtns_static5(rolls, N, no_short, resid5, param5, ff5_cov):
    rtns_static = []
    for i in range(190):
        resid_i = resid5[i*500:i*500+N, :]
        cov_i = np.cov(resid_i)
        cov_i = np.diag(np.diag(cov_i))
        #cov_i[np.where(abs(cov_i)<1e-7)]=0.0
        param5_i = param5[i*500:i*500+N, :]
        ff_vol_i = ff5_cov[i*5:i*5+5, :]
        cov_i = np.matrix(cov_i)
        param5_i = np.matrix(param5_i)
        ff_vol_i = np.matrix(ff_vol_i)
        rtn_cov_i =  param5_i * ff_vol_i * param5_i.T + cov_i
        #st_idx = i*1281
        #ed_idx = i*1281+1260
        #res_mat = rolls[st_idx:ed_idx, 1:N+1]
        #cov_mat =  np.cov(np.array(res_mat).transpose())
        ws_i = get_ws_cov(rtn_cov_i, no_short=no_short)
        st_idx = (i+1)*1281-21
        ed_idx = (i+1)*1281
        res_pre_i = rolls[st_idx:ed_idx, 1:N+1]
        res_pre_i = np.matrix(res_pre_i).T
        res_i = get_return(ws_i, res_pre_i)
        rtns_static += res_i
    return rtns_static

In [106]:
resid51 = np.load('./ff_model_data_1260/residual_mat5.npy')
param51 = np.load('./ff_model_data_1260/params_mat5.npy')
ff1_cov51 = np.load('./ff_model_data_1260/ff5_vol.npy')

In [170]:
N = 100
no_short = False
rtns_static_100 = get_rtns_static5(rolls, N, no_short, resid51, param51, ff1_cov51)
indicators(rtns_static_100)

(10.13412, 15.08486, 0.67181)

In [169]:
N = 200
no_short = False
rtns_static_200 = get_rtns_static5(rolls, N, no_short, resid51, param51, ff1_cov51)
indicators(rtns_static_200)

(11.19787, 15.11662, 0.74077)

In [131]:
N = 500
no_short = False
rtns_static_500 = get_rtns_static5(rolls, N, no_short, resid51, param51, ff1_cov51)
indicators(rtns_static_500)

(11.22358, 14.47839, 0.7752)

## 2.2 one-factor model

In [143]:
def get_rtns_static1(rolls, N, no_short, resid1, param1, ff1_cov):
    rtns_static = []
    for i in range(190):
        resid_i = resid1[i*500:i*500+N, :]
        cov_i = np.cov(resid_i)
        cov_i = np.diag(np.diag(cov_i))
        #cov_i[np.where(abs(cov_i)<1e-7)]=0.0
        param1_i = param1[i*500:i*500+N, :]
        ff_vol_i = ff1_cov[i, :]
        cov_i = np.matrix(cov_i)
        param1_i = np.matrix(param1_i)
        ff_vol_i = np.matrix(ff_vol_i)
        rtn_cov_i =  param1_i * ff_vol_i * param1_i.T + cov_i
        #st_idx = i*1281
        #ed_idx = i*1281+1260
        #res_mat = rolls[st_idx:ed_idx, 1:N+1]
        #cov_mat =  np.cov(np.array(res_mat).transpose())
        ws_i = get_ws_cov(rtn_cov_i, no_short=no_short)
        st_idx = (i+1)*1281-21
        ed_idx = (i+1)*1281
        res_pre_i = rolls[st_idx:ed_idx, 1:N+1]
        res_pre_i = np.matrix(res_pre_i).T
        res_i = get_return(ws_i, res_pre_i)
        rtns_static += res_i
    return rtns_static

In [160]:
resid1 = np.load('./ff_model_data/residual_mat1.npy')
param1 = np.load('./ff_model_data/params_mat1.npy')
ff1_cov1 = np.load('./ff_model_data/ff1_vol.npy')

In [161]:
N = 100
no_short = False
rtns_static1_100 = get_rtns_static1(rolls, N, no_short, resid1, param1, ff1_cov1)
indicators(rtns_static1_100)

(6.61992, 16.61774, 0.39836)

In [141]:
N = 200
no_short = False
rtns_static1_200 = get_rtns_static1(rolls, N, no_short, resid1, param1, ff1_cov1)
indicators(rtns_static1_200)

(8.08311, 16.56508, 0.48796)

In [142]:
N = 500
no_short = False
rtns_static1_500 = get_rtns_static1(rolls, N, no_short, resid1, param1, ff1_cov1)
indicators(rtns_static1_500)

(9.495, 15.83557, 0.5996)

In [162]:
resid11 = np.load('./ff_model_data_1260/residual_mat1.npy')
param11 = np.load('./ff_model_data_1260/params_mat1.npy')
ff1_cov11 = np.load('./ff_model_data_1260/ff1_vol.npy')

In [163]:
N = 100
no_short = False
rtns_static1_1260_100 = get_rtns_static1(rolls, N, no_short, resid11, param11, ff1_cov11)
indicators(rtns_static1_1260_100)

(10.02253, 16.0114, 0.62596)

In [164]:
N = 200
no_short = False
rtns_static1_1260_200 = get_rtns_static1(rolls, N, no_short, resid11, param11, ff1_cov11)
indicators(rtns_static1_1260_200)

(11.2105, 16.31887, 0.68697)

In [165]:
N = 500
no_short = False
rtns_static1_1260_500 = get_rtns_static1(rolls, N, no_short, resid11, param11, ff1_cov11)
indicators(rtns_static1_1260_500)

(11.07083, 16.18345, 0.68408)

# 3 DCC-NL approxiamate model

## 3.1 five factor model

In [39]:
def get_dcc_ff5_res(root_dir, rolls, N, param5, ff5_cov, no_short):
    file_ls = os.listdir(root_dir)
    length = len(file_ls)
    res_dccnl = []
    for i in range(length):
        file_path = os.path.join(root_dir, '{}.csv'.format(i+1))
        resid_cov = pd.read_csv(file_path)
        resid_cov = np.matrix(resid_cov.to_numpy())
        param5_i = param5[i*500:i*500+N, :]
        ff_vol_i = ff5_cov[i*5:i*5+5, :]
        param5_i = np.matrix(param5_i)
        ff_vol_i = np.matrix(ff_vol_i)
        
        rtn_cov_i =  param5_i * ff_vol_i * param5_i.T + resid_cov
        #st_idx = i*1281
        #ed_idx = i*1281+1260
        #res_mat = rolls[st_idx:ed_idx, 1:N+1]
        #cov_mat =  np.cov(np.array(res_mat).transpose())
        ws_i = get_ws_cov(rtn_cov_i, no_short=no_short)
        st_idx = (i+1)*1281-21
        ed_idx = (i+1)*1281
        res_pre_i = rolls[st_idx:ed_idx, 1:N+1]
        res_pre_i = np.matrix(res_pre_i).T
        res_i = get_return(ws_i, res_pre_i)
        res_dccnl += res_i
    return res_dccnl

In [225]:
param5 = np.load('./ff_model_data/params_mat5.npy')
ff5_cov = np.load('./ff_model_data/ff5_vol_dcc.npy')

In [226]:
N = 100
root_dir = './ff_model_data/resid_dcc_cov5_100/'
no_short=False
res_dccnl_ff5_100 = get_dcc_ff5_res(root_dir, rolls, N, param5, ff5_cov, no_short)
indicators(res_dccnl_ff5_100)

(7.47377, 14.23098, 0.52518)

In [227]:
N = 200
root_dir = './ff_model_data/resid_dcc_cov5_200/'
no_short=False
res_dccnl_ff5_200 = get_dcc_ff5_res(root_dir, rolls, N, param5, ff5_cov, no_short)
indicators(res_dccnl_ff5_200)

(9.40873, 13.06866, 0.71995)

In [230]:
param51 = np.load('./ff_model_data_1260/params_mat5.npy')
ff5_cov1 = np.load('./ff_model_data_1260/ff5_vol_dcc.npy')

In [253]:
N = 100
root_dir = './ff_model_data_1260/resid_dcc_cov5_100/'
no_short=False
res_dccnl_ff5_1260_100 = get_dcc_ff5_res(root_dir, rolls, N, param51, ff5_cov1, no_short)
indicators(res_dccnl_ff5_1260_100)

(2.22973, 15.14322, 0.14724)

In [254]:
print(indicators(res_dccnl_ff5_100[0:len(res_dccnl_ff5_1260_100)]))
print(indicators(res_dccnl_100[0:len(res_dccnl_ff5_1260_100)]))

(1.41123, 14.96321, 0.09431)
(-1.71877, 15.24589, -0.11274)


# 3.2 one-factor model

In [63]:
def get_dcc_ff1_res(root_dir, rolls, N, param1, ff1_cov, no_short):
    file_ls = os.listdir(root_dir)
    length = len(file_ls)
    res_dccnl = []
    for i in range(length):
        file_path = os.path.join(root_dir, '{}.csv'.format(i+1))
        resid_cov = pd.read_csv(file_path)
        resid_cov = np.matrix(resid_cov.to_numpy())
        #resid_cov = np.diag(np.diag(resid_cov))
        param1_i = param1[i*500:i*500+N, :]
        ff_vol_i = ff1_cov[i, :]
        resid_cov = np.matrix(resid_cov)
        param5_i = np.matrix(param1_i)
        ff_vol_i = np.matrix(ff_vol_i)
        rtn_cov_i =  param1_i * ff_vol_i * param1_i.T + resid_cov
        #st_idx = i*1281
        #ed_idx = i*1281+1260
        #res_mat = rolls[st_idx:ed_idx, 1:N+1]
        #cov_mat =  np.cov(np.array(res_mat).transpose())
        ws_i = get_ws_cov(rtn_cov_i, no_short=no_short)
        st_idx = (i+1)*1281-21
        ed_idx = (i+1)*1281
        res_pre_i = rolls[st_idx:ed_idx, 1:N+1]
        res_pre_i = np.matrix(res_pre_i).T
        res_i = get_return(ws_i, res_pre_i)
        res_dccnl += res_i
    return res_dccnl

In [ ]:
param1 = np.load('./ff_model_data/params_mat1.npy')
ff1_cov = np.load('./ff_model_data/ff1_vol_dcc.npy')

In [64]:
N = 100
root_dir = './ff_model_data/resid_dcc_cov1_100/'
no_short=False
res_dccnl_ff1_100 = get_dcc_ff1_res(root_dir, rolls, N, param5, ff1_cov, no_short)
indicators(res_dccnl_ff1_100)

(7.86306, 14.23637, 0.55232)

In [90]:
N = 200
root_dir = './ff_model_data/resid_dcc_cov1_200/'
no_short=False
param5 = np.load('./ff_model_data/params_mat1.npy')
ff1_cov = np.load('./ff_model_data/ff1_vol_dcc.npy')
res_dccnl_ff1_200 = get_dcc_ff1_res(root_dir, rolls, N, param5, ff1_cov, no_short)
indicators(res_dccnl_ff1_200)

(9.21941, 13.09512, 0.70403)

# one-factor model 1260

In [167]:
param11 = np.load('./ff_model_data_1260/params_mat1.npy')
ff1_cov1 = np.load('./ff_model_data_1260/ff1_vol_dcc.npy')

In [197]:
N = 100
root_dir = './ff_model_data_1260/resid_dcc_cov1_100/'
no_short=False
res_dccnl_ff1_1260_100 = get_dcc_ff1_res(root_dir, rolls, N, param11, ff1_cov1, no_short)
indicators(res_dccnl_ff1_1260_100)

(9.26, 13.63718, 0.67903)

In [221]:
N = 200
root_dir = './ff_model_data_1260/resid_dcc_cov1_200/'
no_short=False
res_dccnl_ff1_1260_200 = get_dcc_ff1_res(root_dir, rolls, N, param11, ff1_cov1, no_short)
indicators(res_dccnl_ff1_1260_200)

(10.06905, 12.38824, 0.81279)